# 1. Authenticate & Import

In [1]:
import ee
# 网不好的时候自动重试ee.Initialize()
print("Try init ee...", end=" ")
eeInited, tryCounts = False, 0
while not eeInited:
    print("%d" % tryCounts, end=" ")
    try:
        ee.Initialize()
    except Exception as e:
        if (str(e)[0:23]=="Please authorize access"):
            ee.Authenticate()
        tryCounts += 1
    else:
        eeInited = True
        print("\nee initialized!")

import geemap

Try init ee... 0 
ee initialized!


In [2]:
import os
import time
import random
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import cv2

import gdal
import utils
import common
import dataset
import GetMapTiles
from models.ResNet import ResNet101

# 2. Import Earth Engine Data

In [3]:
# Boundary and Grid-----------------------------------------------------------------------------------------------------
worldBoundary = ee.FeatureCollection("users/liuph/shape/WorldBoundary")
ChinaBoundary = ee.FeatureCollection("users/410093033/China")
WorldGrid5d = ee.FeatureCollection("users/liuph/shape/WorldGrid5dC5")

# Fusion of landcover products------------------------------------------------------------------------------------------
# forest_fuse = ee.ImageCollection('users/sysushiqian/forestFuse2020').min().rename("forest_fuse")
forest_fuse = ee.ImageCollection('users/sysushiqian/forestFuse2020_new').min().rename("ForestFuse")
forest23 = forest_fuse.expression('b(0)>=2 && b(0)<=3')

# 3. Needed Functions

In [4]:
def checkStatus(dataPath, predPath, totalList):
    predList = [int(item.name.split(".")[0].split("_")[-1]) for item in os.scandir(predPath)]
    
    unProcessedList = list(set(totalList) - set(predList) - set(geList))
    return len(predList), len(geList), len(unProcessedList)

# 3. Download Google Earth(GE) image and Forest Fusion(FF) image

In [5]:
totalGrid = ee.FeatureCollection("users/sunly3456/globalGrids/globalGt60Grids/forest23gt60gridsAll")
totalGridIdxList = list(range(1, 100))

dataPath = r"E:/SZT/Data/Forest/densePredData"
predPath = r"./densePred"

maxMaintainNum = 100 # 维持不超过maxMaintainNum数量的影像

# 按totalGridIdxList中的索引，依次预测对应0.1度格网
for gridIdx in totalGridIdxList:
    while(True):
        geList = [int(item.name.split(".")[0].split("_")[-1]) for item in os.scandir(dataPath) if item.name.split("_")[0] == "GE17"]
        maintainNum = len(geList)
        if maintainNum >= maxMaintainNum:
            print("### data storage reached limit, waiting for 1 sec...")
            time.sleep(60)
        else: 
            break

    feature01 = totalGrid.filterMetadata("orderID", "equals", gridIdx).first()
    ID_5d = feature01.getNumber("gridID").getInfo()

    fNamePostfix = "_%d_%d.tif" % (ID_5d, gridIdx)

    curFFPath = dataPath + "/FF01" + fNamePostfix
    curGEPath = dataPath + "/GE17" + fNamePostfix
    predSavePath = predPath + "/densePred" + fNamePostfix

    print("\n\n[Grid5d=%d Grid01d=%d] %s" % (ID_5d, gridIdx, time.ctime()))
    print("%s" % "-"*80)
  
    if os.path.exists(predSavePath): # 如果已经预测过，则跳过
        print("already predicted, jump to next")
        continue
    else: # 如果还未预测，则检查是否需要下载
        if not os.path.exists(curGEPath):
            curCoord = np.array(feature01.geometry().coordinates().getInfo()[0])
            xmin01, ymin01 = np.min(curCoord[:, 0]), np.min(curCoord[:, 1])
            padding = 0.002
            print("# Google Earth Image: ")
            GetMapTiles.getpic_tif(xmin01-padding, ymin01+0.1+padding, xmin01+0.1+padding, ymin01-padding, 17, source='google', out_filename=curGEPath, style='s')
        else:
            print("%s already exits" % curGEPath.split('/')[-1])
            
        if not os.path.exists(curFFPath):
            print("# Forest Fusion Image: ")
            geemap.ee_export_image(forest_fuse, curFFPath, scale=30, region=feature01.geometry())
        else:
            print("%s already exits" % curFFPath.split('/')[-1])




[Grid5d=525 Grid01d=1] Mon Dec 28 15:39:58 2020
--------------------------------------------------------------------------------
already predicted, jump to next


[Grid5d=525 Grid01d=2] Mon Dec 28 15:39:58 2020
--------------------------------------------------------------------------------
already predicted, jump to next


[Grid5d=525 Grid01d=3] Mon Dec 28 15:39:59 2020
--------------------------------------------------------------------------------
already predicted, jump to next


[Grid5d=525 Grid01d=4] Mon Dec 28 15:40:00 2020
--------------------------------------------------------------------------------
already predicted, jump to next


[Grid5d=525 Grid01d=5] Mon Dec 28 15:40:00 2020
--------------------------------------------------------------------------------
already predicted, jump to next


[Grid5d=525 Grid01d=6] Mon Dec 28 15:40:01 2020
--------------------------------------------------------------------------------
already predicted, jump to next


[Grid5d=525 Grid01d=